##### U-Net architecture

See [here](https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py#L19) for code and [here](https://arxiv.org/pdf/1505.04597.pdf) for the original literature.

In [1]:
def restart_kernel(restart=False):
    import IPython
    app = IPython.Application.instance()
    if restart:
        app.kernel.do_shutdown(True)

restart_kernel(False)

import os
os.environ["THEANO_FLAGS"] = "device=cpu,lib.cnmem=0"

In [18]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Concatenate, Activation, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, ZeroPadding2D, Cropping2D, BatchNormalization, Dropout
from keras.optimizers import Adadelta, Adam
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras import backend as K
#from keras.regularizers import l2
#from keras.constraints import maxnorm
#from keras.layers.advanced_activations import LeakyReLU

#keras_backend.set_data_format('channels_first')  # Theano dimension ordering in this code
# "tf" assumes (rows, cols, channels) while "th" assumes (channels, rows, cols)
# Possibly change this around natively in the data so the backend doesn't have to switch them
# Only necessary if I use TF!

import skimage.exposure
import cv2

from matplotlib import pyplot as plt
from pushbullet import Pushbullet

### Helper functions

In [3]:
# Pushbullet notifier
def push(title='Done!',text=''):
    Pushbullet('o.YFPNNPfGRekivaCGHa4qMSgjZt8zJ6FL').devices[0].push_note(title,text)
    
# Import the training data
def import_data(img_h,img_w):
    
    x_train = np.load('./data/training-data/tiled_patched_images_576_576_train.npy')
    x_test = np.load('./data/training-data/tiled_patched_images_576_576_test.npy')
    y_train = np.load('./data/training-data/tiled_patched_masks_576_576_train.npy')
    y_test = np.load('./data/training-data/tiled_patched_masks_576_576_test.npy')
    
    #train_imgs = np.load('./data/training-data/resized_data_{}x{}_imgs.npy'.format(img_w,img_h))[()]
    #train_msks = np.load('./data/training-data/resized_data_{}x{}_msks.npy'.format(img_w,img_h))[()]
    
    #x = np.rollaxis(np.concatenate([i[np.newaxis,...] for i in train_imgs.values()],axis=0),3,1)
    #y = np.rollaxis(np.concatenate([i[np.newaxis,...] for i in train_msks.values()],axis=0),3,1).astype(np.float32)
    #y = y.reshape((y.shape[0],y.shape[1],y.shape[2]*y.shape[3]))
    '''
    Classes:
    0 Buildings - large building, residential, non-residential, fuel storage facility, fortified building
    1 Misc. Manmade structures 
    2 Road 
    3 Track - poor/dirt/cart track, footpath/trail
    4 Trees - woodland, hedgerows, groups of trees, standalone trees
    5 Crops - contour ploughing/cropland, grain (wheat) crops, row (potatoes, turnips) crops
    6 Waterway 
    7 Standing water
    8 Vehicle Large - large vehicle (e.g. lorry, truck,bus), logistics vehicle
    9 Vehicle Small - small vehicle (car, van), motorbike
    '''    
    return x_train,x_test, y_train,y_test

img_h,img_w = 160,160
x_train,x_test, y_train,y_test = import_data(img_h,img_w)

# Increment the counter
def counter():
    run = np.load('./data/misc/run_counter.npy')
    run += 1
    np.save('./data/misc/run_counter.npy',run)
    return run
run = counter()

# Set the counter to a specific value
def set_counter(run):
    run = run
    np.save('./data/misc/run_counter.npy',run)
    return run
# Uncomment the next line to manually set the counter if something goes wrong
run = set_counter(127)
print('This is run # %i' %run)

This is run # 127


In [22]:
%%time
def compiler(img_rows = x_train.shape[2],img_cols = x_train.shape[3],
            nfilters = 64, act = 'relu',init = 'he_normal',
            lr=0.001,decay=0.0,rate=[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2],transpose=True):
    
    def jaccard(y_true, y_pred,smooth=1.):
        #y_true_f = K.flatten(y_true)
        #y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true * y_pred)
        return (intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) - intersection + smooth)
    
    def Conv2DReluBatchNorm(n_filter, filter_size, layer_in, activation, kernel_initializer='he_uniform'):
        return BatchNormalization(axis=1)(Activation(activation=act)((Conv2D(n_filter, filter_size, padding='same',kernel_initializer=init)(layer_in))))
        
    def up_conv(nfilters,filter_factor,layer_in,kernel_initializer=init,activation=act):
        if transpose:
            return BatchNormalization(axis=1)(Activation(activation=act)(Conv2D(nfilters*filter_factor, (2, 2), padding='same',kernel_initializer=init)(Conv2DTranspose(nfilters*filter_factor,2,2,strides=(2, 2),padding='same',kernel_initializer=init,activation='relu')(layer_in))))
        else:
            return BatchNormalization(axis=1)(Activation(activation=act)(Conv2D(nfilters*filter_factor, (2, 2), padding='same',kernel_initializer=init)(UpSampling2D(size=(2, 2))(layer_in))))
        
    inputs = Input((20, img_rows, img_cols))
    
    conv1 = Conv2DReluBatchNorm(nfilters, (3, 3), inputs, activation=act,kernel_initializer=init)
    conv1 = Conv2DReluBatchNorm(nfilters, (3, 3), conv1, activation=act,kernel_initializer=init)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(rate=rate[0])(pool1)

    conv2 = Conv2DReluBatchNorm(nfilters*2, (3, 3), pool1, activation=act,kernel_initializer=init)
    conv2 = Conv2DReluBatchNorm(nfilters*2, (3, 3), conv2, activation=act,kernel_initializer=init)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(rate=rate[1])(pool2)

    conv3 = Conv2DReluBatchNorm(nfilters*4, (3, 3), pool2, activation=act,kernel_initializer=init)
    conv3 = Conv2DReluBatchNorm(nfilters*4, (3, 3), conv3, activation=act,kernel_initializer=init)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(rate=rate[2])(pool3)

    conv4 = Conv2DReluBatchNorm(nfilters*8, (3, 3), pool3, activation=act,kernel_initializer=init)
    conv4 = Conv2DReluBatchNorm(nfilters*8, (3, 3), conv4, activation=act,kernel_initializer=init)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(rate=rate[3])(pool4)

    conv5 = Conv2DReluBatchNorm(nfilters*16, (3, 3), pool4, activation=act,kernel_initializer=init)
    conv5 = Conv2DReluBatchNorm(nfilters*16, (3, 3), conv5, activation=act,kernel_initializer=init)
    conv5 = Dropout(rate=rate[4])(conv5)
        
    up6 = Concatenate(axis=1)([up_conv(nfilters,8,conv5), conv4])
    #up6 = merge([up_conv(nfilters,8,conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Conv2DReluBatchNorm(nfilters*8, (3, 3), up6, activation=act,kernel_initializer=init)
    conv6 = Conv2DReluBatchNorm(nfilters*8, (3, 3), conv6, activation=act,kernel_initializer=init)
    conv6 = Dropout(rate=rate[5])(conv6)
    
    up7 = Concatenate(axis=1)([up_conv(nfilters,8,conv6), conv3])
    #up7 = merge([up_conv(nfilters,4,conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Conv2DReluBatchNorm(nfilters*4, (3, 3), up7, activation=act,kernel_initializer=init)
    conv7 = Conv2DReluBatchNorm(nfilters*4, (3, 3), conv7, activation=act,kernel_initializer=init)
    conv7 = Dropout(rate=rate[6])(conv7)
    
    up8 = Concatenate(axis=1)([up_conv(nfilters,2,conv7), conv2])
    #up8 = merge([up_conv(nfilters,2,conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Conv2DReluBatchNorm(nfilters*2, (3, 3), up8, activation=act,kernel_initializer=init)
    conv8 = Conv2DReluBatchNorm(nfilters*2, (3, 3), conv8, activation=act,kernel_initializer=init)
    conv8 = Dropout(rate=rate[7])(conv8)

    up9 = Concatenate(axis=1)([up_conv(nfilters,1,conv8), conv1])
    #up9 = merge([up_conv(nfilters,1,conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Conv2DReluBatchNorm(nfilters, (3, 3), up9, activation=act,kernel_initializer=init)
    conv9 = Conv2DReluBatchNorm(nfilters, (3, 3), conv9, activation=act,kernel_initializer=init)
    conv9 = Dropout(rate=rate[8])(conv9)
    
    conv10 = Conv2DReluBatchNorm(1, (1, 1), conv9, activation=act,kernel_initializer=init)
    output = Activation(activation='sigmoid')(conv10)
    
    model = Model(inputs=inputs, outputs=output)
    
    model.compile(optimizer=Adam(lr=lr,decay=decay), loss='binary_crossentropy',metrics=[jaccard])

    return model

#rate=[0]*9
rate=[0.1,0.2,0.3,0.4,0.5,0.4,0.3,0.2,0.1] # current version
#rate=[0.2,0.3,0.4,0.5,0.5,0.5,0.4,0.3,0.2] # symmetric but more dropout
#rate=[0.2,0.2,0.3,0.3,0.4,0.4,0.5,0.5,0.6] # increasing

model = compiler(img_rows=x_train.shape[2],img_cols=x_train.shape[3],
            nfilters=32,act='relu',init='he_normal',
            lr=0.001,rate=rate,transpose=False)
model.save_weights('./data/misc/initial_weights.hdf5')
#model.summary()

CPU times: user 5.63 s, sys: 55.9 ms, total: 5.69 s
Wall time: 5.74 s


In [ ]:
def trainer(model,class_,epochs=1000,fit=True,use_existing=False):
    print('This is run # {} for class {}'.format(run,class_))
    
    if use_existing:
        model.load_weights('./data/weights/model_weights_class_{}_run_{}.hdf5'.format(class_,run))
        
    if fit:
        quitter = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=50, verbose=1, mode='auto')
        lrreducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, verbose=1, mode='auto', epsilon=0.001, cooldown=2, min_lr=0)
        model_checkpoint = ModelCheckpoint('./data/weights/model_weights_run_{}_class_{}.hdf5'.format(run,class_), monitor='val_loss', save_best_only=True)
        csvlogger = CSVLogger('./data/logs/training_log_run_{}_class_{}'.format(run,class_), separator=',', append=True)
        # tensorboard = TensorBoard(log_dir='./data/logs/'+'tensorboard_all-classes-run_{:04d}'.format(run), histogram_freq=0, write_graph=True, write_images=False)
        # tensorboard --logdir=data/logs
        
        model.fit(x_train, y_train[:,class_:class_+1,:,:],
                  batch_size=5,
                  epochs=epochs,
                  verbose=1,
                  shuffle=True,
                  callbacks=[model_checkpoint,csvlogger,quitter,lrreducer],
                  validation_data=(x_test,y_test[:,class_:class_+1,:,:]),
                  initial_epoch=0)
            
    preds = model.predict(x_train, verbose=1)
    np.save('./data/predictions/predictions_run_{}_class_{}.npy'.format(run,class_), preds)
    
    return model

for class_ in range(3,4):
    model.load_weights('./data/misc/initial_weights.hdf5')
    model = trainer(model,class_,fit=True,use_existing=False)
    model.save('./data/models/u-net-complete-model-run_{}_class_{}.hdf5'.format(run,class_))
    push('Training on class {} is done'.format(class_),
     'Train loss: %f, train jaccard: %f, val loss %f, val jaccard%f' %(model.history.history['loss'][-1],model.history.history['jaccard'][-1],model.history.history['val_loss'][-1],model.history.history['val_jaccard'][-1]))

This is run # 127 for class 3
